In [4]:
import deepchem as dc
from openpom.feat.graph_featurizer import GraphFeaturizer, GraphConvConstants
from openpom.utils.data_utils import get_class_imbalance_ratio, IterativeStratifiedSplitter
from openpom.models.mpnn_pom import MPNNPOMModel
from datetime import datetime
from tqdm import tqdm
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

In [6]:
def x_y_split(df):
  """                                                                                                                     Splies the oncoming dataset to X and
  y for classification.                                                                                                 
  :param df: A molecular dataset for odor prediction
  :type df: pandas Dataframe
  :return: A list of classes/labels for each row.
  :rtype: pandas dataframes                                                                                               """
  try:
    x = df[['IsomericSMILES', 'CID']].copy()
  except:
     x = df['IsomericSMILES'].copy()
  try:
      y = df.drop(['IsomericSMILES', 'Descriptors', 'CID', 'Descriptor Count'], axis=1).copy()
  except:
      try:
          y = df.drop(['IsomericSMILES', 'Descriptors', 'Descriptor Count'], axis=1).copy()
      except:
        try:
          y = df.drop(['IsomericSMILES', 'Descriptors', 'CID'], axis=1).copy()
        except:
          y = df.drop(['IsomericSMILES', 'Descriptors'], axis=1).copy()
  return x, y

In [ ]:
'''
TASKS = [
'alcoholic', 'aldehydic', 'alliaceous', 'almond', 'amber', 'animal',
'anisic', 'apple', 'apricot', 'aromatic', 'balsamic', 'banana', 'beefy',
'bergamot', 'berry', 'bitter', 'black currant', 'brandy', 'burnt',
'buttery', 'cabbage', 'camphoreous', 'caramellic', 'cedar', 'celery',
'chamomile', 'cheesy', 'cherry', 'chocolate', 'cinnamon', 'citrus', 'clean',
'clove', 'cocoa', 'coconut', 'coffee', 'cognac', 'cooked', 'cooling',
'cortex', 'coumarinic', 'creamy', 'cucumber', 'dairy', 'dry', 'earthy',
'ethereal', 'fatty', 'fermented', 'fishy', 'floral', 'fresh', 'fruit skin',
'fruity', 'garlic', 'gassy', 'geranium', 'grape', 'grapefruit', 'grassy',
'green', 'hawthorn', 'hay', 'hazelnut', 'herbal', 'honey', 'hyacinth',
'jasmin', 'juicy', 'ketonic', 'lactonic', 'lavender', 'leafy', 'leathery',
'lemon', 'lily', 'malty', 'meaty', 'medicinal', 'melon', 'metallic',
'milky', 'mint', 'muguet', 'mushroom', 'musk', 'musty', 'natural', 'nutty',
'odorless', 'oily', 'onion', 'orange', 'orangeflower', 'orris', 'ozone',
'peach', 'pear', 'phenolic', 'pine', 'pineapple', 'plum', 'popcorn',
'potato', 'powdery', 'pungent', 'radish', 'raspberry', 'ripe', 'roasted',
'rose', 'rummy', 'sandalwood', 'savory', 'sharp', 'smoky', 'soapy',
'solvent', 'sour', 'spicy', 'strawberry', 'sulfurous', 'sweaty', 'sweet',
'tea', 'terpenic', 'tobacco', 'tomato', 'tropical', 'vanilla', 'vegetable',
'vetiver', 'violet', 'warm', 'waxy', 'weedy', 'winey', 'woody'
]

print("No of tasks: ", len(TASKS))
n_tasks = len(TASKS)
'''

No of tasks:  138


save train and test splits

In [1]:
# # uncomment and run if no splits saved yet

# # download curated dataset
# !wget https://raw.githubusercontent.com/ARY2260/openpom/main/openpom/data/curated_datasets/curated_GS_LF_merged_4983.csv

# # The curated dataset can also found at `openpom/data/curated_datasets/curated_GS_LF_merged_4983.csv` in the repo.

# input_file = 'curated_GS_LF_merged_4983.csv' # or new downloaded file path

# # get dataset

# featurizer = GraphFeaturizer()
# smiles_field = 'nonStereoSMILES'
# loader = dc.data.CSVLoader(tasks=TASKS,
#                    feature_field=smiles_field,
#                    featurizer=featurizer)
# dataset = loader.create_dataset(inputs=[input_file])
# n_tasks = len(dataset.tasks)

# # get train valid test splits
# splitter = IterativeStratifiedSplitter(order=2)
# train_dataset, test_dataset = splitter.train_test_split(dataset, frac_train=0.8, train_dir='./splits/train_data', test_dir='./splits/test_data')

# print("train_dataset: ", len(train_dataset))
# print("test_dataset: ", len(test_dataset))

In [27]:
train = pd.read_csv('../data/splits/alldesc_train_2025.csv')
test = pd.read_csv('../data/splits/alldesc_test_2025.csv')

train_x, train_y = x_y_split(train)
test_x, test_y = x_y_split(test)

In [28]:
TASKS = train_y.columns.values.tolist()
print("No of tasks: ", len(TASKS))
n_tasks = len(TASKS)

No of tasks:  146


In [29]:
featurizer = GraphFeaturizer()
smiles_field = 'IsomericSMILES'
loader = dc.data.CSVLoader(tasks=TASKS,
                   feature_field=smiles_field,
                   featurizer=featurizer)
train_dataset = loader.create_dataset('../data/splits/alldesc_train_2025.csv')
test_dataset = loader.create_dataset('../data/splits/alldesc_test_2025.csv')
n_tasks = len(train_dataset.tasks)

print("train_dataset: ", len(train_dataset))
print("test_dataset: ", len(test_dataset))

train_dataset:  5331
test_dataset:  1380


In [30]:
train_dataset.move('../data/splits/train_data')
test_dataset.move('../data/splits/test_data')

load splits

In [31]:
train_dataset = dc.data.DiskDataset('../data/splits/train_data')
test_dataset = dc.data.DiskDataset('../data/splits/test_data')
print("train_dataset: ", len(train_dataset))
print("test_dataset: ", len(test_dataset))

train_dataset:  5331
test_dataset:  1380


set parameters

In [32]:
train_ratios = get_class_imbalance_ratio(train_dataset)
assert len(train_ratios) == n_tasks

# learning_rate = 0.001
learning_rate = dc.models.optimizers.ExponentialDecay(initial_rate=0.001, decay_rate=0.5, decay_steps=32*20, staircase=True)

metric = dc.metrics.Metric(dc.metrics.roc_auc_score)

In [7]:
# run this cell if detailed log is needed

# import logging

# logger = logging.getLogger(__name__)
# logging.basicConfig(level=logging.INFO)

In [36]:
# no of models in the ensemble
n_models = 2

# no of epochs each model is trained for
nb_epoch = 62

In [37]:
for i in tqdm(range(n_models)):
    model = MPNNPOMModel(n_tasks = n_tasks,
                            batch_size=128,
                            learning_rate=learning_rate,
                            class_imbalance_ratio = train_ratios,
                            loss_aggr_type = 'sum',
                            node_out_feats = 100,
                            edge_hidden_feats = 75,
                            edge_out_feats = 100,
                            num_step_message_passing = 5,
                            mpnn_residual = True,
                            message_aggregator_type = 'sum',
                            mode = 'classification',
                            number_atom_features = GraphConvConstants.ATOM_FDIM,
                            number_bond_features = GraphConvConstants.BOND_FDIM,
                            n_classes = 1,
                            readout_type = 'set2set',
                            num_step_set2set = 3,
                            num_layer_set2set = 2,
                            ffn_hidden_list= [392, 392],
                            ffn_embeddings = 256,
                            ffn_activation = 'relu',
                            ffn_dropout_p = 0.12,
                            ffn_dropout_at_input_no_act = False,
                            weight_decay = 1e-5,
                            self_loop = False,
                            optimizer_name = 'adam',
                            log_frequency = 32,
                            model_dir = f'./ensemble_models/experiments_{i+1}',
                            device_name='cpu')

    start_time = datetime.now()
    
    # fit model
    loss = model.fit(
          train_dataset,
          nb_epoch=nb_epoch,
          max_checkpoints_to_keep=1,
          deterministic=False,
          restore=False)
    end_time = datetime.now()
    
    train_scores = model.evaluate(train_dataset, [metric])['roc_auc_score']
    test_scores = model.evaluate(test_dataset, [metric])['roc_auc_score']
    print(f"loss = {loss}; train_scores = {train_scores}; test_scores = {test_scores}; time_taken = {str(end_time-start_time)}")
    model.save_checkpoint() # saves final checkpoint => `checkpoint2.pt`
    del model
    torch.cuda.empty_cache()

  0%|          | 0/2 [02:35<?, ?it/s]


KeyboardInterrupt: 

Get test score from the ensemble

In [9]:
list_preds = []
for i in range(n_models):
    model = MPNNPOMModel(n_tasks = n_tasks,
                            batch_size=128,
                            learning_rate=learning_rate,
                            class_imbalance_ratio = train_ratios,
                            loss_aggr_type = 'sum',
                            node_out_feats = 100,
                            edge_hidden_feats = 75,
                            edge_out_feats = 100,
                            num_step_message_passing = 5,
                            mpnn_residual = True,
                            message_aggregator_type = 'sum',
                            mode = 'classification',
                            number_atom_features = GraphConvConstants.ATOM_FDIM,
                            number_bond_features = GraphConvConstants.BOND_FDIM,
                            n_classes = 1,
                            readout_type = 'set2set',
                            num_step_set2set = 3,
                            num_layer_set2set = 2,
                            ffn_hidden_list= [392, 392],
                            ffn_embeddings = 256,
                            ffn_activation = 'relu',
                            ffn_dropout_p = 0.12,
                            ffn_dropout_at_input_no_act = False,
                            weight_decay = 1e-5,
                            self_loop = False,
                            optimizer_name = 'adam',
                            log_frequency = 32,
                            model_dir = f'./ensemble_models/experiments_{i+1}',
                            device_name='cuda')
    model.restore(f"./ensemble_models/experiments_{i+1}/checkpoint2.pt")
    # test_scores = model.evaluate(test_dataset, [metric])['roc_auc_score']
    # print("test_score: ", test_scores)
    preds = model.predict(test_dataset)
    list_preds.append(preds)

preds_arr = np.asarray(list_preds)
ensemble_preds = np.mean(preds_arr, axis=0)
print("average ensemble score: ", roc_auc_score(test_dataset.y, ensemble_preds, average="macro"))

average ensemble score:  0.9332425000551015
